In [1]:
%store -r f_list

no stored variable f_list


In [3]:
# !pip install lightgbm

    100% |████████████████████████████████| 1.2MB 25.3MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!pip install tensorflow_hub

    100% |████████████████████████████████| 71kB 25.0MB/s ta 0:00:01
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import time

In [6]:
import tensorflow as tf
import tensorflow_hub as hub

%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib.tensorboard.plugins import projector

In [7]:
header = ['query_id', 'query', 'passage_text', 'label', 'passage_id']
test_header = ['query_id', 'query', 'passage_text', 'passage_id']

In [8]:
# train = pd.read_csv('./data/data.tsv',delimiter='\t',names=header)

In [11]:
test = pd.read_csv('../search_data/eval1_unlabelled.tsv',delimiter='\t',names=test_header)

In [49]:
train

,query_id,query,passage_text,label,passage_id
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4
5,131,. what is a corporation?,McDonald's Corporation is one of the most reco...,1,5
6,131,. what is a corporation?,Corporations are owned by their stockholders (...,0,6
7,131,. what is a corporation?,An Association is an organized group of people...,0,7
8,131,. what is a corporation?,B Corp certification shines a light on the com...,0,8
9,131,. what is a corporation?,LLCs offer greater flexibility when it comes t...,0,9


In [12]:
test

,query_id,query,passage_text,passage_id
0,1135787,distance between erie in buffalo new york,Erie Canal Distance Tables The Erie Canal is t...,0
1,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,1
2,1135787,distance between erie in buffalo new york,The distance between Erie and Buffalo in a str...,2
3,1135787,distance between erie in buffalo new york,Erie Canal Distances. Erie Canal Distance Tabl...,3
4,1135787,distance between erie in buffalo new york,Erie's Metropolitan Area consists of approxima...,4
5,1135787,distance between erie in buffalo new york,This will help you estimate how much time you ...,5
6,1135787,distance between erie in buffalo new york,"As a general guide, Albany to Oswego, NY will ...",6
7,1135787,distance between erie in buffalo new york,"Erie, PA. {{::location.tagLine.value.text}}. C...",7
8,1135787,distance between erie in buffalo new york,"You start your flight Distance from Buffalo, N...",8
9,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,9


## Uni Sentence Encoder

In [5]:
def cos_sim(a, b):
	"""Takes 2 vectors a, b and returns the cosine similarity according 
	to the definition of the dot product
	"""
	dot_product = np.dot(a, b)
	norm_a = np.linalg.norm(a)
	norm_b = np.linalg.norm(b)
	return dot_product / (norm_a * norm_b)

In [6]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder-large/3"

In [7]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module('../models',trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.


In [8]:
embed.variable_map

{'Embeddings_en/sharded_0': <tf.Variable 'module/Embeddings_en/sharded_0:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_1': <tf.Variable 'module/Embeddings_en/sharded_1:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_2': <tf.Variable 'module/Embeddings_en/sharded_2:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_3': <tf.Variable 'module/Embeddings_en/sharded_3:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_4': <tf.Variable 'module/Embeddings_en/sharded_4:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_5': <tf.Variable 'module/Embeddings_en/sharded_5:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_6': <tf.Variable 'module/Embeddings_en/sharded_6:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_7': <tf.Variable 'module/Embeddings_en/sharded_7:0' shape=(35297, 320) dtype=float32_ref>,
 'Embeddings_en/sharded_8': <tf.Variable 'module/Embeddings_en/sharded_8:0' shap

## Elmo Model

In [13]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/elmo/2'.
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 150.35MB
INFO:tensorflow:Downloading https://tfhub.dev/google/elmo/2: 330.35MB
INFO:tensorflow:Downloaded https://tfhub.dev/google/elmo/2, Total size: 357.40MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/elmo/2'.


In [ ]:
elmo.

In [14]:
embeddings = elmo(["Argentina played football very well","Brazil is a strong team", 
                   "Artists all over the world are attending the play", 
                   "Child is playing the guitar", 
                   "There was absolute silence during the play"],
             signature="default",
             as_dict=True)["elmo"]

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [15]:
embeddings

<tf.Tensor 'module_apply_default/aggregation/mul_3:0' shape=(5, 9, 1024) dtype=float32>

In [16]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)


Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [17]:
embeddings

<tf.Tensor 'module_apply_default/aggregation/mul_3:0' shape=(5, 9, 1024) dtype=float32>

In [18]:
print(sess.run(embeddings[0][0]))

[-0.5879137  -0.02317354 -0.13742939 ...  0.0367917   0.17688549
  0.15371276]


In [20]:
print(sess.run(embeddings[1]))

[[-0.44688168 -0.25053233  0.0726297  ...  0.00752543  0.26000363
   0.07398823]
 [-0.55936646 -0.25759098 -0.22243893 ... -0.03210151  0.9437138
   0.60099435]
 [-0.06743803  0.04363684  0.14544795 ...  0.18022753  0.13792133
   0.16251738]
 ...
 [-0.02840839 -0.04353216  0.04130163 ...  0.02583167 -0.01429833
  -0.01650422]
 [-0.02840839 -0.04353216  0.04130163 ...  0.02583167 -0.01429833
  -0.01650422]
 [-0.02840839 -0.04353216  0.04130163 ...  0.02583167 -0.01429833
  -0.01650422]]


In [9]:
tf.logging.set_verbosity(tf.logging.ERROR)
session = tf.Session()

In [10]:
session.run([tf.global_variables_initializer(), tf.tables_initializer()])

[None, None]

In [24]:
train.loc[0]['query']

'. what is a corporation?'

In [27]:
embed([train.loc[0]['query']])

<tf.Tensor 'module_apply_default/Encoder_en/hidden_layers/l2_normalize:0' shape=(1, 512) dtype=float32>

In [28]:
qu = session.run(embed([train.loc[0]['query']]))

In [29]:
qu

array([[ 3.40036186e-03, -1.47261135e-02,  8.58818963e-02,
        -6.97190091e-02, -2.67279204e-02, -5.08100837e-02,
         1.96543783e-02,  3.63989174e-02, -4.18017469e-02,
        -6.53274655e-02, -2.79143881e-02,  5.95376156e-02,
         8.98364410e-02,  5.17266840e-02,  4.78091538e-02,
        -5.44558279e-02, -3.09981145e-02,  4.79324162e-02,
        -3.55827319e-03, -1.38094081e-02,  1.54041965e-02,
         3.67238410e-02, -9.58832800e-02,  1.12534417e-02,
         8.18369910e-03, -8.71178042e-03, -1.57915521e-02,
         4.35859635e-02, -4.00554724e-02, -7.26192966e-02,
        -2.95748599e-02, -7.94395804e-02, -3.86218354e-02,
         3.15692909e-02,  1.03715668e-03, -8.49271938e-03,
         1.82631630e-02,  4.74782055e-03,  1.07522905e-01,
         2.99583580e-02,  2.49413382e-02, -2.75488179e-02,
        -2.65536308e-02,  2.42663901e-02,  4.90826219e-02,
         4.56498675e-02, -7.55163431e-02,  8.39870702e-03,
         5.12238452e-03, -1.60691142e-02,  9.16647315e-0

In [40]:
messages = list(train.loc[0:9]['passage_text'])

In [41]:
messages

['A company is incorporated in a specific nation, often within the bounds of a smaller subset of that nation, such as a state or province. The corporation is then governed by the laws of incorporation in that state. A corporation may issue stock, either private or public, or may be classified as a non-stock corporation. If stock is issued, the corporation will usually be governed by its shareholders, either directly or indirectly.',
 'Today, there is a growing community of more than 2,100 Certified B Corps from 50 countries and over 130 industries working together toward 1 unifying goal: to redefine success in business. Join the Movement',
 'Corporation definition, an association of individuals, created by law or under authority of law, having a continuous existence independent of the existences of its members, and powers and liabilities distinct from those of its members. See more.',
 'Examples of corporation in a Sentence. 1  He works as a consultant for several large corporations. 2

In [42]:
res = session.run(embed(messages))

In [43]:
res

array([[-0.01746481, -0.01268308,  0.09080733, ...,  0.03637503,
        -0.07537873,  0.08401209],
       [-0.08236408, -0.02815819,  0.03551569, ...,  0.02062435,
        -0.1085479 ,  0.04510164],
       [ 0.01079089, -0.00369102,  0.09665293, ...,  0.05551924,
        -0.07355407,  0.08441055],
       ...,
       [-0.06552306,  0.0341304 ,  0.06541747, ...,  0.00969458,
        -0.0817094 ,  0.0743005 ],
       [ 0.04275434, -0.01085981,  0.07364381, ...,  0.02097764,
        -0.11303937,  0.04835946],
       [-0.04342113,  0.00611222,  0.04029054, ...,  0.0170644 ,
        -0.05315609,  0.0762021 ]], dtype=float32)

In [47]:
index=0
for row in res:
    print (index, cos_sim(qu,row))
    index+=1

0 [0.6503212]
1 [0.41656098]
2 [0.7142911]
3 [0.57499516]
4 [0.59456366]
5 [0.68104386]
6 [0.6480441]
7 [0.53973436]
8 [0.39962935]
9 [0.46747363]


In [48]:
train2 = train.copy()

In [52]:
# q_embeddings = session.run(embed(list(train['query'])))

KeyboardInterrupt: 

In [50]:
q_embeddings

array([[-0.01746481, -0.01268308,  0.09080733, ...,  0.03637503,
        -0.07537873,  0.08401209],
       [-0.08236408, -0.02815819,  0.03551569, ...,  0.02062435,
        -0.1085479 ,  0.04510164],
       [ 0.01079089, -0.00369102,  0.09665293, ...,  0.05551924,
        -0.07355407,  0.08441055],
       ...,
       [-0.06552306,  0.0341304 ,  0.06541747, ...,  0.00969458,
        -0.0817094 ,  0.0743005 ],
       [ 0.04275434, -0.01085981,  0.07364381, ...,  0.02097764,
        -0.11303937,  0.04835946],
       [-0.04342113,  0.00611222,  0.04029054, ...,  0.0170644 ,
        -0.05315609,  0.0762021 ]], dtype=float32)

In [51]:
ans_embeddings = session.run(embed(list(train['passage_text'])))

0                             . what is a corporation?
1                             . what is a corporation?
2                             . what is a corporation?
3                             . what is a corporation?
4                             . what is a corporation?
5                             . what is a corporation?
6                             . what is a corporation?
7                             . what is a corporation?
8                             . what is a corporation?
9                             . what is a corporation?
10                           symptoms of a dying mouse
11                           symptoms of a dying mouse
12                           symptoms of a dying mouse
13                           symptoms of a dying mouse
14                           symptoms of a dying mouse
15                           symptoms of a dying mouse
16                           symptoms of a dying mouse
17                           symptoms of a dying mouse
18        

In [53]:
q_eval_embeddings = session.run(embed(list(test['query'])))

In [54]:
q_eval_embeddings[2]

array([-9.02520716e-02, -1.55035229e-02, -2.85901446e-02, -3.81376478e-03,
        8.71831551e-02, -5.80941997e-02, -2.21908968e-02,  1.29015278e-02,
       -3.27162780e-02,  6.29166961e-02, -6.55514281e-03,  1.10093560e-02,
       -4.68899421e-02,  1.73011925e-02,  6.81616887e-02,  5.78744058e-03,
        3.75055782e-02,  4.60489541e-02, -3.29235382e-02,  4.02892493e-02,
        7.81329945e-02, -1.68268867e-02, -2.85828323e-03, -2.18836702e-02,
       -8.65548477e-02, -8.39050710e-02,  5.58573641e-02,  9.95650329e-03,
       -3.52650434e-02,  2.27832831e-02,  4.74609360e-02,  4.48822565e-02,
        2.78441645e-02, -2.76789954e-03, -5.20821474e-02, -3.47533450e-02,
       -2.44764388e-02,  2.62407642e-02,  8.87815468e-03, -1.54638961e-02,
       -6.27869070e-02,  7.64529034e-02,  2.58646794e-02, -4.62735109e-02,
        5.10799699e-02,  1.36161624e-02, -5.55705540e-02, -3.18440273e-02,
       -2.40788367e-02, -2.10575536e-02,  1.57752447e-02,  4.64527914e-03,
       -3.43705304e-02,  

In [11]:
## causing kernael to restart, doing one by one
# ans_eval_embeddings = session.run(embed(list(test['passage_text'])))
ans_eval_embeddings = []
start_time = time.time()

ans_eval_embeddings = embed(list(test['passage_text']))

# index=1
# for row in list(test['passage_text']):
# #     print (row)
# #     ans_eval_embeddings.append(session.run(embed([row])))
#     ans_eval_embeddings.append(embed([row]))
# #     print (ans_eval_embeddings)
# #     print (type(ans_eval_embeddings), type(ans_eval_embeddings[0]))
#     if index%100 == 0:
#         print (index, time.time()-start_time)
#     index+=1
print (time.time()-start_time)
print ((ans_eval_embeddings))

27.9645414352417
Tensor("module_apply_default/Encoder_en/hidden_layers/l2_normalize:0", shape=(104170, 512), dtype=float32)


In [12]:
a = ans_eval_embeddings[0]

In [13]:
a

<tf.Tensor 'strided_slice:0' shape=(512,) dtype=float32>

In [ ]:
# a = ans_eval_embeddings[0]
# b = ans_eval_embeddings[1]
# start_time = time.time()
# normalize_a = tf.nn.l2_normalize(a,0)        
# normalize_b = tf.nn.l2_normalize(b,0)
# cos_similarity=tf.reduce_sum(tf.multiply(normalize_a,normalize_b))
# cos_sim=session.run(cos_similarity)
# print (time.time()-start_time)
# print (cos_sim)

In [ ]:
ans_eval_embeddings_mat = session.run(a)

In [ ]:
# ans_eval_embeddings[0].eval(session=session)

In [18]:
from multiprocessing.pool import ThreadPool


In [39]:
def create_numpy_array(row):
    start_time = time.time()
    
#     session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    tensor = embed([row])
    embeddings_row = session.run(tensor)
    print (time.time()-start_time)
    return embeddings_row

In [40]:
def fetch_embeddings(text_rows, thread_num = 8):
    datalist = []
    pool = ThreadPool(thread_num)
#     for i in range(0, len(req_list), step):
#     req_list_list = divide_into_list_of_list(req_list,step)
    list_of_list_of_data = pool.map(create_numpy_array, text_rows)
    for single_list in list_of_list_of_data:
        datalist.extend(single_list)
        
    pool.close()
    pool.join()
    #print type(results)
    return datalist

In [41]:
dat = list(test[:20]['passage_text'])

In [42]:
session = tf.Session()
session.run([tf.global_variables_initializer(), tf.tables_initializer()])

[None, None]

In [43]:
datalist = fetch_embeddings(dat,1)

RuntimeError: Module must be applied in the graph it was instantiated for.

In [ ]:
start_time = time.time()
# ans_eval_embeddings_mat = ans_eval_embeddings.eval(session=session)
# session.run(ans_eval_embeddings)
print (time.time()-start_time)

In [28]:
cos_sim(a[0][0],a[1][0])

0.697469

## Cosine similarity

## Light GBM

### Tf-Idf

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [45]:
nltk_stop_words = set(["i","me","my","myself","we","our","ours","ourselves","you","you're","you've","you'll","you'd",
                       "your","yours","yourself","yourselves","he","him","his","himself","she","she's","her","hers",
                       "herself","it","it's","its","itself","they","them","their","theirs","themselves","what","which",
                       "who","whom","this","that","that'll","these","those","am","is","are","was","were","be","been",
                       "being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if",
                       "or","because","as","until","while","of","at","by","for","with","about","against","between",
                       "into","through","during","before","after","above","below","to","from","up","down","in","out",
                       "on","off","over","under","again","further","then","once","here","there","when","where","why",
                       "how","all","any","both","each","few","more","most","other","some","such","no","nor","not",
                       "only","own","same","so","than","too","very","s","t","can","will","just","don","don't","should",
                       "should've","now","d","ll","m","o","re","ve","y","ain","aren","aren't","couldn","couldn't","didn",
                       "didn't","doesn","doesn't","hadn","hadn't","hasn","hasn't","haven","haven't","isn","isn't","ma",
                       "mightn","mightn't","mustn","mustn't","needn","needn't","shan","shan't","shouldn","shouldn't",
                       "wasn","wasn't","weren","weren't","won","won't","wouldn","wouldn't"])


In [94]:
vectorizer = TfidfVectorizer(ngram_range=(1,1),stop_words=nltk_stop_words,max_features=20000)

In [51]:
train

,query_id,query,passage_text,label,passage_id
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4
5,131,. what is a corporation?,McDonald's Corporation is one of the most reco...,1,5
6,131,. what is a corporation?,Corporations are owned by their stockholders (...,0,6
7,131,. what is a corporation?,An Association is an organized group of people...,0,7
8,131,. what is a corporation?,B Corp certification shines a light on the com...,0,8
9,131,. what is a corporation?,LLCs offer greater flexibility when it comes t...,0,9


In [52]:
train['combined'] = train['query'] + " " + train['passage_text']

In [54]:
test['combined'] = test['query'] + " " + test['passage_text']

#### functions

In [56]:
import re

In [57]:
# Cleaning level-1
def clean_delimiters(line):
    line = str(line)
    ## \xa0 is non-breaking space in Latin1 (ISO 8859-1), also chr(160)
    line = line.replace("\r", " ").replace("\\r", " ").replace("\n", " ").replace(
        "\\n", " ").replace("\t", " ").replace("\\t", " ").replace("\"", "").replace(
        ",", " ").replace("|", " ").replace(":", " ").replace("--", "").replace(u'\xa0', u' ').replace("<br>", "")
    return re.sub(" +"," ",line)

def small_case(line):
    line = str(line)
    return line.lower()
#     sd.Description.str.lower()


In [75]:
import nltk

In [79]:
# import nltk.punkt
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize

ps = PorterStemmer()

In [80]:
ps.stem("munchies")

'munchi'

In [81]:
nltk.download('punkt'

SyntaxError: unexpected EOF while parsing (<ipython-input-81-11c516ae986e>, line 1)

In [84]:
def stemming(line):
    words = line.split()
    stemmed = [ps.stem(w) for w in words]
    return " ".join(stemmed)

In [85]:
stemming("sv hi is this")

'sv hi is thi'

In [58]:
train['combined'] = train['combined'].apply(small_case)
train['combined'] = train['combined'].apply(clean_delimiters)

In [59]:
test['combined'] = test['combined'].apply(small_case)
test['combined'] = test['combined'].apply(clean_delimiters)

In [86]:
train['combined'] = train['combined'].apply(stemming)


In [87]:
test['combined'] = test['combined'].apply(stemming)


In [88]:
train

,query_id,query,passage_text,label,passage_id,combined
0,131,. what is a corporation?,A company is incorporated in a specific nation...,0,0,. what is a corporation? a compani is incorpor...
1,131,. what is a corporation?,"Today, there is a growing community of more th...",0,1,. what is a corporation? today there is a grow...
2,131,. what is a corporation?,"Corporation definition, an association of indi...",0,2,. what is a corporation? corpor definit an ass...
3,131,. what is a corporation?,Examples of corporation in a Sentence. 1 He w...,0,3,. what is a corporation? exampl of corpor in a...
4,131,. what is a corporation?,1: a government-owned corporation (as a utilit...,0,4,. what is a corporation? 1 a government-own co...
5,131,. what is a corporation?,McDonald's Corporation is one of the most reco...,1,5,. what is a corporation? mcdonald' corpor is o...
6,131,. what is a corporation?,Corporations are owned by their stockholders (...,0,6,. what is a corporation? corpor are own by the...
7,131,. what is a corporation?,An Association is an organized group of people...,0,7,. what is a corporation? an associ is an organ...
8,131,. what is a corporation?,B Corp certification shines a light on the com...,0,8,. what is a corporation? b corp certif shine a...
9,131,. what is a corporation?,LLCs offer greater flexibility when it comes t...,0,9,. what is a corporation? llc offer greater fle...


In [61]:
test

,query_id,query,passage_text,label,passage_id,combined
0,1135787,distance between erie in buffalo new york,Erie Canal Distance Tables The Erie Canal is t...,0,NaN,distance between erie in buffalo new york erie...
1,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,1,NaN,distance between erie in buffalo new york what...
2,1135787,distance between erie in buffalo new york,The distance between Erie and Buffalo in a str...,2,NaN,distance between erie in buffalo new york the ...
3,1135787,distance between erie in buffalo new york,Erie Canal Distances. Erie Canal Distance Tabl...,3,NaN,distance between erie in buffalo new york erie...
4,1135787,distance between erie in buffalo new york,Erie's Metropolitan Area consists of approxima...,4,NaN,distance between erie in buffalo new york erie...
5,1135787,distance between erie in buffalo new york,This will help you estimate how much time you ...,5,NaN,distance between erie in buffalo new york this...
6,1135787,distance between erie in buffalo new york,"As a general guide, Albany to Oswego, NY will ...",6,NaN,distance between erie in buffalo new york as a...
7,1135787,distance between erie in buffalo new york,"Erie, PA. {{::location.tagLine.value.text}}. C...",7,NaN,distance between erie in buffalo new york erie...
8,1135787,distance between erie in buffalo new york,"You start your flight Distance from Buffalo, N...",8,NaN,distance between erie in buffalo new york you ...
9,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,9,NaN,distance between erie in buffalo new york what...


In [95]:
vectorizer.fit(train['combined'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=20000, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words={'ain', 'she', 'have', 'didn', 'her', "you've", "aren't", 'or', 'haven', 'most', 'your', "don't", 'no', 're', 'where', "you'd", 'own', 'out', 'while', 'isn', 'am', 'themselves', 'only', 'further', 'them', 'that', 'more', 'needn', 'himself', 'are', 'd', 'doing', "should've", 'couldn', 'at'...gh', 'there', "mightn't", 'him', 'shan', 'and', 'a', 'will', 'same', 'other', 'these', 'll', 'then'},
        strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [96]:
len(vectorizer.vocabulary_)

20000

In [97]:
X = vectorizer.transform(train['combined'])

In [100]:
X.shape

(5241880, 20000)

In [101]:
X_test = vectorizer.transform(test['combined'])

In [102]:
X_test.shape


(104170, 20000)

### First trying with normal binary classification and creating file in required format

In [108]:
q_train = []
q_test = []

for i in range(0,X.shape[0],10):
    q_train.append(10)
    
for i in range(0,X_test.shape[0],10):
    q_test.append(10)

if len(q_train)*10 != X.shape[0]:
    print ("Wrong")
    
if len(q_test)*10 != X_test.shape[0]:
    print ("Wrong test size")

In [112]:
header

['query_id', 'query', 'passage_text', 'label', 'passage_id']

In [114]:
y_train = list(train['label'])

In [ ]:
def save_eval_file():
    

In [193]:
gbdt = lgb.LGBMClassifier(boosting_type='gbdt',n_estimators=600,num_round=300,
                     learning_rate=0.3,num_leaves=31,
                     n_jobs=16,silent=False)

In [194]:
gbdt.fit(X,y_train)

/codemill/choudhaa/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.3, max_depth=-1,
        min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=600, n_jobs=16, num_leaves=31, num_round=300,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=False, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

### LambdaRank now

In [161]:
%matplotlib inline

In [176]:
gbm = lgb.LGBMRanker(boosting_type='gbdt',n_estimators=800,num_round=400,#label_gain=ad,
                     max_position=10,
                     learning_rate=0.25,num_leaves=31,
                     n_jobs=16,silent=False)

In [177]:
## query group format is 10
gbm.fit(X, y_train, group=q_train, #eval_set=[(X_test, y_test_2)],
        #eval_group=[q_test], eval_at=[5, 10], early_stopping_rounds=100,
        verbose=2)




/codemill/choudhaa/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


LGBMRanker(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
      importance_type='split', learning_rate=0.25, max_depth=-1,
      max_position=10, min_child_samples=20, min_child_weight=0.001,
      min_split_gain=0.0, n_estimators=800, n_jobs=16, num_leaves=31,
      num_round=400, objective=None, random_state=None, reg_alpha=0.0,
      reg_lambda=0.0, silent=False, subsample=1.0,
      subsample_for_bin=200000, subsample_freq=0)

In [195]:
gbm2 = gbm

In [196]:
gbm = gbdt

In [197]:
y_pred = gbm.predict(X_test,group=q_test)

/codemill/choudhaa/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [198]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [199]:
len(y_pred)

104170

In [200]:
min(y_pred)

0

In [201]:
mini = min(y_pred)
y_pred_2 = [x-mini for x in y_pred]
y_pred_2[0], min(y_pred_2)

(0, 0)

In [140]:
# ans = test.copy()

In [202]:
ans

,query_id,query,passage_text,passage_id,label
0,1135787,distance between erie in buffalo new york,Erie Canal Distance Tables The Erie Canal is t...,0,0.023093
1,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,1,0.030071
2,1135787,distance between erie in buffalo new york,The distance between Erie and Buffalo in a str...,2,0.273480
3,1135787,distance between erie in buffalo new york,Erie Canal Distances. Erie Canal Distance Tabl...,3,0.061335
4,1135787,distance between erie in buffalo new york,Erie's Metropolitan Area consists of approxima...,4,0.168945
5,1135787,distance between erie in buffalo new york,This will help you estimate how much time you ...,5,-0.104056
6,1135787,distance between erie in buffalo new york,"As a general guide, Albany to Oswego, NY will ...",6,0.151875
7,1135787,distance between erie in buffalo new york,"Erie, PA. {{::location.tagLine.value.text}}. C...",7,0.096746
8,1135787,distance between erie in buffalo new york,"You start your flight Distance from Buffalo, N...",8,-0.155854
9,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,9,0.030071


In [184]:
# for i in range(0,X_test.shape[0],10):
#     q_test.append(10)

In [203]:
ans['label'] = y_pred_2
ans['label'] = y_pred

In [204]:
ans

,query_id,query,passage_text,passage_id,label
0,1135787,distance between erie in buffalo new york,Erie Canal Distance Tables The Erie Canal is t...,0,0
1,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,1,0
2,1135787,distance between erie in buffalo new york,The distance between Erie and Buffalo in a str...,2,0
3,1135787,distance between erie in buffalo new york,Erie Canal Distances. Erie Canal Distance Tabl...,3,0
4,1135787,distance between erie in buffalo new york,Erie's Metropolitan Area consists of approxima...,4,0
5,1135787,distance between erie in buffalo new york,This will help you estimate how much time you ...,5,0
6,1135787,distance between erie in buffalo new york,"As a general guide, Albany to Oswego, NY will ...",6,0
7,1135787,distance between erie in buffalo new york,"Erie, PA. {{::location.tagLine.value.text}}. C...",7,0
8,1135787,distance between erie in buffalo new york,"You start your flight Distance from Buffalo, N...",8,0
9,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,9,0


In [143]:
def softmax(X, theta = 1.0, axis = None):
    """
    Compute the softmax of each element along an axis of X.

    Parameters
    ----------
    X: ND-Array. Probably should be floats. 
    theta (optional): float parameter, used as a multiplier
        prior to exponentiation. Default = 1.0
    axis (optional): axis to compute values along. Default is the 
        first non-singleton axis.

    Returns an array the same size as X. The result will sum to 1
    along the specified axis.
    """

    # make X at least 2d
    y = np.atleast_2d(X)

    # find axis
    if axis is None:
        axis = next(j[0] for j in enumerate(y.shape) if j[1] > 1)

    # multiply y against the theta parameter, 
    y = y * float(theta)

    # subtract the max for numerical stability
    y = y - np.expand_dims(np.max(y, axis = axis), axis)
    
    # exponentiate y
    y = np.exp(y)

    # take the sum along the specified axis
    ax_sum = np.expand_dims(np.sum(y, axis = axis), axis)

    # finally: divide elementwise
    p = y / ax_sum

    # flatten if X was 1D
    if len(X.shape) == 1: p = p.flatten()

    return p

In [144]:
ans

,query_id,query,passage_text,passage_id,label
0,1135787,distance between erie in buffalo new york,Erie Canal Distance Tables The Erie Canal is t...,0,1.313704
1,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,1,1.276414
2,1135787,distance between erie in buffalo new york,The distance between Erie and Buffalo in a str...,2,1.407472
3,1135787,distance between erie in buffalo new york,Erie Canal Distances. Erie Canal Distance Tabl...,3,1.323222
4,1135787,distance between erie in buffalo new york,Erie's Metropolitan Area consists of approxima...,4,1.449395
5,1135787,distance between erie in buffalo new york,This will help you estimate how much time you ...,5,1.202849
6,1135787,distance between erie in buffalo new york,"As a general guide, Albany to Oswego, NY will ...",6,1.500592
7,1135787,distance between erie in buffalo new york,"Erie, PA. {{::location.tagLine.value.text}}. C...",7,1.414031
8,1135787,distance between erie in buffalo new york,"You start your flight Distance from Buffalo, N...",8,1.303376
9,1135787,distance between erie in buffalo new york,What is the distance between Erie AND Buffalo?...,9,1.276414


In [205]:
ans_headers = ['query_id','1','2','3','4','5','6','7','8','9','10']
ans_eval = pd.DataFrame(columns=ans_headers)

In [206]:
ans_eval

,query_id,1,2,3,4,5,6,7,8,9,10


In [207]:
for i in range(0,int(len(y_pred_2)/10)):
    row = [ans.iloc[i*10]['query_id']]
    for j in range(i*10,i*10+10):
        row.append(ans.iloc[j]['label'])
    ans_eval.loc[i] = row

In [209]:
ans_eval.to_csv('./data/answer.tsv',sep='\t',header=False,index=False)#,compression='zip')

In [ ]:

10
10
10

### Decreasing learning_rate
gbm.fit(X_train, y_train, group=q_train, eval_set=[(X_test, y_test)],
        eval_group=[q_test], eval_at=[1, 5], early_stopping_rounds=10,
        verbose=2,
        callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.95 ** x * 0.1),
                  lgb.print_evaluation(period=0, show_stdv=True)])

## LightFM

In [210]:
from lightfm import LightFM

In [214]:
item_features = X
user_features = 1

In [212]:
ITEM_ALPHA = 0.1
NUM_COMPONENTS = 50

### create train matrix

In [217]:
matrix = []

In [218]:
for i in range(0,int(len(y_train)/10)):
    row = []
    for j in range(i*10,i*10+10):
        row.append(train.iloc[j]['label'])
    matrix.append(row)

In [ ]:
matrix_test = []
for i in range(0,int(len(y_test)/10)):
    row = np.zeros(10)
    matrix_test.append(row)

In [213]:
# Define a new model instance
model = LightFM(loss='warp',
                item_alpha=ITEM_ALPHA,
                no_components=NUM_COMPONENTS)

In [ ]:
model = model.fit(matrix,
                item_features=item_features,
                epochs=NUM_EPOCHS,
                num_threads=NUM_THREADS)

In [ ]:
model